In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/merged_aadhaar_data_sample.csv')


In [4]:
df.describe

<bound method NDFrame.describe of                date                        state        district  pincode  \
0        2025-03-01    Andaman & Nicobar Islands        Andamans   744101   
1        2025-03-01  Andaman and Nicobar Islands         Nicobar   744301   
2        2025-03-01  Andaman and Nicobar Islands         Nicobar   744302   
3        2025-03-01  Andaman and Nicobar Islands         Nicobar   744303   
4        2025-03-01  Andaman and Nicobar Islands         Nicobar   744304   
...             ...                          ...             ...      ...   
2330394  2025-12-31                  West Bengal  West Midnapore   721426   
2330395  2025-12-31                  West Bengal  West Midnapore   721504   
2330396  2025-12-31                  West Bengal  West Midnapore   721506   
2330397  2025-12-31                  West Bengal  West Midnapore   721507   
2330398  2025-12-31                  West Bengal  West Midnapore   721517   

         age_0_5  age_5_17  age_18_greate

In [5]:
df['state'].unique()

array(['Andaman & Nicobar Islands', 'Andaman and Nicobar Islands',
       'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chandigarh', 'Chhattisgarh', 'Dadra & Nagar Haveli',
       'Dadra and Nagar Haveli',
       'Dadra and Nagar Haveli and Daman and Diu', 'Daman & Diu',
       'Daman and Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka',
       'Kerala', 'Ladakh', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra',
       'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Orissa',
       'Pondicherry', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim',
       'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh',
       'Uttarakhand', 'West Bengal',
       'The Dadra And Nagar Haveli And Daman And Diu',
       'Jammu And Kashmir', 'Jammu & Kashmir', 'ODISHA', 'WEST BENGAL',
       'WESTBENGAL', 'West  Bengal', 'West bengal', 'Westbengal',
       'andhra pradesh', 'odisha', 'west Bengal', '100000', 'We

In [8]:
import re

STATE_MAPPING = {
    "andaman & nicobar islands": "Andaman and Nicobar Islands",
    "andaman and nicobar islands": "Andaman and Nicobar Islands",

    "dadra & nagar haveli": "Dadra and Nagar Haveli and Daman and Diu",
    "dadra and nagar haveli": "Dadra and Nagar Haveli and Daman and Diu",
    "dadra and nagar haveli and daman and diu": "Dadra and Nagar Haveli and Daman and Diu",
    "the dadra and nagar haveli and daman and diu": "Dadra and Nagar Haveli and Daman and Diu",
    "daman & diu": "Dadra and Nagar Haveli and Daman and Diu",
    "daman and diu": "Dadra and Nagar Haveli and Daman and Diu",

    "odisha": "Odisha",
    "orissa": "Odisha",

    "west bengal": "West Bengal",
    "west  bengal": "West Bengal",
    "westbengal": "West Bengal",
    "west bangal": "West Bengal",
    "west bengli": "West Bengal",

    "jammu and kashmir": "Jammu and Kashmir",
    "jammu & kashmir": "Jammu and Kashmir",

    "chhattisgarh": "Chhattisgarh",
    "chhatisgarh": "Chhattisgarh",

    "pondicherry": "Puducherry",
    "puducherry": "Puducherry",

    "andhra pradesh": "Andhra Pradesh",
    "tamil nadu": "Tamil Nadu",
    "tamilnadu": "Tamil Nadu",
    "uttaranchal": "Uttarakhand",
    "uttarakhand": "Uttarakhand",
}

def _normalize_key(s: str) -> str:
    """
    Internal helper: lowercases, strips, collapses spaces, removes some punctuation
    to create a stable key for STATE_MAPPING lookup.
    """
    if s is None:
        return ""
    s = str(s)
    s = s.strip()
    # replace multiple spaces with single space
    s = re.sub(r"\s+", " ", s)
    # remove common punctuation that doesn't change meaning
    s = re.sub(r"[.,'\"!?/\\;-]", "", s)
    return s.lower()

def standardize_state_name(s: str) -> str:
    """
    Turn raw state string into a cleaned, canonical form.
    - strips whitespace
    - collapses internal spaces
    - applies mapping
    - otherwise returns title-cased version
    """
    if pd.isna(s):
        return s  # keep NaN as NaN

    raw = str(s)
    raw_stripped = re.sub(r"\s+", " ", raw.strip())

    key = _normalize_key(raw_stripped)

    # First try explicit mapping
    if key in STATE_MAPPING:
        return STATE_MAPPING[key]

    # Otherwise, just normalize casing: "WEST BENGAL" -> "West Bengal"
    return raw_stripped.title()


In [10]:
df["state"] = df["state"].apply(standardize_state_name)
df["state"].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal', '100000',
       'Balanagar', 'Darbhanga', 'Puttenahalli', 'Nagpur', 'Jaipur',
       'Raja Annamalai Puram', 'Madanapalle'], dtype=object)

In [13]:
VALID_STATES = {
    "Andhra Pradesh",
    "Arunachal Pradesh",
    "Assam",
    "Bihar",
    "Chhattisgarh",
    "Goa",
    "Gujarat",
    "Haryana",
    "Himachal Pradesh",
    "Jharkhand",
    "Karnataka",
    "Kerala",
    "Madhya Pradesh",
    "Maharashtra",
    "Manipur",
    "Meghalaya",
    "Mizoram",
    "Nagaland",
    "Odisha",
    "Punjab",
    "Rajasthan",
    "Sikkim",
    "Tamil Nadu",
    "Telangana",
    "Tripura",
    "Uttar Pradesh",
    "Uttarakhand",
    "West Bengal",
    # Union Territories
    "Andaman and Nicobar Islands",
    "Chandigarh",
    "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi",  # or "NCT of Delhi" depending on your canonical choice
    "Jammu and Kashmir",
    "Ladakh",
    "Lakshadweep",
    "Puducherry",
}

In [14]:
df["state"] = df["state"].apply(standardize_state_name)
df["state"].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal', '100000',
       'Balanagar', 'Darbhanga', 'Puttenahalli', 'Nagpur', 'Jaipur',
       'Raja Annamalai Puram', 'Madanapalle'], dtype=object)

In [15]:
# Boolean flag for rows whose state is not in the official list
df["state_valid"] = df["state"].isin(VALID_STATES)

# Quick counts
valid_counts = df["state_valid"].value_counts(dropna=False)
print(valid_counts)

state_valid
True     2330369
False         30
Name: count, dtype: int64


In [16]:
invalid_states = sorted(df.loc[~df["state_valid"], "state"].dropna().unique())
invalid_states

['100000',
 'Balanagar',
 'Darbhanga',
 'Jaipur',
 'Madanapalle',
 'Nagpur',
 'Puttenahalli',
 'Raja Annamalai Puram']

In [17]:
df = df[df["state_valid"]].copy()
df.shape

(2330369, 12)

In [18]:
df['state'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'],
      dtype=object)